## Tutorial: Stitching Networks between Chameleon Sites

This notebook demonstrates how to use ExoGENI to stitch networks between Chameleon sites. It will walk through reserving and configuring stitchable networks on each Chameleon site and using the xoStitch tool to create a layer2 circuit between the networks. 

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [1]:
import json
import os
import chi

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project
chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name

#Insert keypair name
key_name = 'pruth-jupyter'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=2

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [2]:
#Set the region
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
uc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(uc_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-07-24T16:52:00.000000",
  "reservations": [
    {
      "status": "pending",
      "lease_id": "9f03cd52-2e42-45c2-b72a-d8605336cdef",
      "resource_id": "eaac1b9a-7365-41c7-8e99-4d326800caf3",
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "created_at": "2020-07-23 16:52:55",
      "updated_at": "2020-07-23 16:52:57",
      "required_floatingips": [],
      "missing_resources": false,
      "amount": 1,
      "id": "72e7f3b4-90dc-4693-82dc-d7fe1b7adf05",
      "resource_type": "virtual:floatingip",
      "resources_changed": false
    },
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "9f03cd52-2e42-45c2-b72a-d8605336cdef",
      "resource_id": "f9c86cc5-e36f-4d90-a7d1-238dcc6662e7",
      "max": 2,
      "created_at": "2020-07-23 16:52:47",
      "min": 2,
      "updated_at": "2020-07-23 16:52:53",
      "missing_

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [3]:
#Get the lease by name
uc_lease = get_lease_by_name(lease_name)

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

uc_compute_reservation_id: e38613bc-58fb-4d5a-b9bd-7fa2bb0fce73
uc_network_reservation_id: f3b32fdd-9f7e-4559-8c3b-e4888b4a110a
uc_floatingip_reservation_id: 72e7f3b4-90dc-4693-82dc-d7fe1b7adf05


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [4]:
chi.set('region_name', 'CHI@UC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
uc_network_vlan = network['provider:segmentation_id']
print('uc_network_vlan: ' + str(uc_network_vlan))


{
  "provider:physical_network": "exogeni",
  "ipv6_address_scope": null,
  "revision_number": 1,
  "port_security_enabled": true,
  "mtu": 1500,
  "id": "e0fa08b8-ee90-4f5d-b4eb-11f60e46ca19",
  "router:external": false,
  "availability_zone_hints": [],
  "availability_zones": [],
  "ipv4_address_scope": null,
  "shared": false,
  "project_id": "d9faac3973a847f1b718fa765fe312e2",
  "status": "ACTIVE",
  "subnets": [],
  "description": "",
  "tags": [],
  "updated_at": "2020-07-23T16:53:10Z",
  "provider:segmentation_id": 3291,
  "name": "pruthNet",
  "admin_state_up": true,
  "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
  "created_at": "2020-07-23T16:53:10Z",
  "provider:network_type": "vlan"
}
Network ID: e0fa08b8-ee90-4f5d-b4eb-11f60e46ca19
uc_network_vlan: 3291


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [5]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

{
  "subnets": [
    {
      "service_types": [],
      "description": "",
      "enable_dhcp": true,
      "tags": [],
      "network_id": "e0fa08b8-ee90-4f5d-b4eb-11f60e46ca19",
      "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
      "created_at": "2020-07-23T16:54:45Z",
      "dns_nameservers": [],
      "updated_at": "2020-07-23T16:54:45Z",
      "gateway_ip": "192.168.42.1",
      "ipv6_ra_mode": null,
      "allocation_pools": [
        {
          "start": "192.168.42.2",
          "end": "192.168.42.254"
        }
      ],
      "host_routes": [],
      "revision_number": 0,
      "ip_version": 4,
      "ipv6_address_mode": null,
      "cidr": "192.168.42.0/24",
      "project_id": "d9faac3973a847f1b718fa765fe312e2",
      "id": "6563631a-90b5-499f-9ecf-7bc9e62aa2eb",
      "subnetpool_id": null,
      "name": "pruthsubnet"
    }
  ]
}


#### Add a Router

TODO: add links here

In [7]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

{
  "router": {
    "status": "ACTIVE",
    "external_gateway_info": {
      "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
      "enable_snat": true,
      "external_fixed_ips": [
        {
          "subnet_id": "c3950603-9e04-4cc5-be8d-1efbfe59fc0a",
          "ip_address": "192.5.87.56"
        }
      ]
    },
    "availability_zone_hints": [],
    "availability_zones": [],
    "description": "",
    "tags": [],
    "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
    "created_at": "2020-07-23T16:55:11Z",
    "admin_state_up": true,
    "updated_at": "2020-07-23T16:55:13Z",
    "flavor_id": null,
    "revision_number": 3,
    "routes": [],
    "project_id": "d9faac3973a847f1b718fa765fe312e2",
    "id": "46bd0078-f99a-49d7-9996-f106a4c9c148",
    "name": "pruthRouter"
  }
}


#### Attach the Router and Subnet

TODO: Add links here

In [8]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': 'e0fa08b8-ee90-4f5d-b4eb-11f60e46ca19',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '6563631a-90b5-499f-9ecf-7bc9e62aa2eb',
 'subnet_ids': ['6563631a-90b5-499f-9ecf-7bc9e62aa2eb'],
 'port_id': 'df921672-95d6-405e-b350-07593bd11c26',
 'id': '46bd0078-f99a-49d7-9996-f106a4c9c148'}

## Create TACC Network and Server

#### Create a Lease at TACC

In [9]:
#Set the region
chi.set('region_name', 'CHI@TACC')     # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(tacc_lease, indent=2))

{
  "status": "PENDING",
  "user_id": "f3a422ce5a4442e3a7a247ddfd98bfc3",
  "name": "pruthLease",
  "end_date": "2020-07-24T16:55:00.000000",
  "reservations": [
    {
      "status": "pending",
      "before_end": "default",
      "lease_id": "f6bdcb5a-f243-4506-895d-1a6ae5c7a42f",
      "resource_id": "39cfca00-c1fc-4a8d-a0c1-2a763f2d8a33",
      "max": 2,
      "created_at": "2020-07-23 16:55:26",
      "min": 2,
      "updated_at": "2020-07-23 16:55:41",
      "missing_resources": false,
      "hypervisor_properties": "",
      "on_start": "default",
      "resource_properties": "[\"==\", \"$node_type\", \"compute_haswell\"]",
      "id": "1ead9202-c138-45b9-b235-ce0bf59f9bcb",
      "resource_type": "physical:host",
      "resources_changed": false
    },
    {
      "status": "pending",
      "lease_id": "f6bdcb5a-f243-4506-895d-1a6ae5c7a42f",
      "resource_id": "97ea7342-72aa-483b-927f-5130d61eebfe",
      "created_at": "2020-07-23 16:55:41",
      "updated_at": "2020-07-23 16

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [10]:
#Get the lease by name
tacc_lease = get_lease_by_name(lease_name)

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

tacc_compute_reservation_id: 1ead9202-c138-45b9-b235-ce0bf59f9bcb
tacc_network_reservation_id: 87996aba-8981-4540-91bb-5df5317d8586
tacc_floatingip_reservation_id: f912ca67-9888-45a8-a6ee-bdcab28d3f5f


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [11]:
chi.set('region_name', 'CHI@TACC')  
#Get the network
network = get_network_by_name(network_name)
print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
tacc_network_vlan = network['provider:segmentation_id']
print('tacc_network_vlan: ' + str(tacc_network_vlan))


{
  "provider:physical_network": "exogeni",
  "ipv6_address_scope": null,
  "revision_number": 1,
  "port_security_enabled": true,
  "mtu": 1500,
  "id": "2d703759-ea1c-449c-ad88-38bb67c588be",
  "router:external": false,
  "availability_zone_hints": [],
  "availability_zones": [],
  "ipv4_address_scope": null,
  "shared": false,
  "project_id": "d9faac3973a847f1b718fa765fe312e2",
  "status": "ACTIVE",
  "subnets": [],
  "description": "",
  "tags": [],
  "updated_at": "2020-07-23T16:56:12Z",
  "provider:segmentation_id": 3504,
  "name": "pruthNet",
  "admin_state_up": true,
  "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
  "created_at": "2020-07-23T16:56:12Z",
  "provider:network_type": "vlan"
}
Network ID: 2d703759-ea1c-449c-ad88-38bb67c588be
tacc_network_vlan: 3504


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [12]:
subnet = add_subnet(subnet_name, network_name, cidr=cidr)

print(json.dumps(subnet, indent=2))

{
  "subnets": [
    {
      "service_types": [],
      "description": "",
      "enable_dhcp": true,
      "tags": [],
      "network_id": "2d703759-ea1c-449c-ad88-38bb67c588be",
      "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
      "created_at": "2020-07-23T16:57:02Z",
      "dns_nameservers": [],
      "updated_at": "2020-07-23T16:57:02Z",
      "gateway_ip": "192.168.42.1",
      "ipv6_ra_mode": null,
      "allocation_pools": [
        {
          "start": "192.168.42.2",
          "end": "192.168.42.254"
        }
      ],
      "host_routes": [],
      "revision_number": 0,
      "ip_version": 4,
      "ipv6_address_mode": null,
      "cidr": "192.168.42.0/24",
      "project_id": "d9faac3973a847f1b718fa765fe312e2",
      "id": "81f39fde-8ad0-475e-a977-f0f2c3f2ec96",
      "subnetpool_id": null,
      "name": "pruthsubnet"
    }
  ]
}


#### Add a Router

TODO: add links here

In [13]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

{
  "router": {
    "status": "ACTIVE",
    "external_gateway_info": {
      "network_id": "6189521e-06a0-4c43-b163-16cc11ce675b",
      "enable_snat": true,
      "external_fixed_ips": [
        {
          "subnet_id": "a58a9684-844d-4424-821a-b1e75ebfc0f7",
          "ip_address": "129.114.109.75"
        }
      ]
    },
    "availability_zone_hints": [],
    "availability_zones": [],
    "description": "",
    "tags": [],
    "tenant_id": "d9faac3973a847f1b718fa765fe312e2",
    "created_at": "2020-07-23T16:57:04Z",
    "admin_state_up": true,
    "updated_at": "2020-07-23T16:57:06Z",
    "flavor_id": null,
    "revision_number": 3,
    "routes": [],
    "project_id": "d9faac3973a847f1b718fa765fe312e2",
    "id": "6337c0ed-60e0-4d66-b04f-ea9a137353cd",
    "name": "pruthRouter"
  }
}


#### Attach the Router and Subnet

TODO: Add links here

In [14]:
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

{'network_id': '2d703759-ea1c-449c-ad88-38bb67c588be',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '81f39fde-8ad0-475e-a977-f0f2c3f2ec96',
 'subnet_ids': ['81f39fde-8ad0-475e-a977-f0f2c3f2ec96'],
 'port_id': 'c7b082dd-cca6-4b1c-8bbd-d142a31bd864',
 'id': '6337c0ed-60e0-4d66-b04f-ea9a137353cd'}

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [15]:
chi.set('region_name', 'CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [18]:
chi.set('region_name', 'CHI@UC')  
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 192.5.87.221


#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [16]:
chi.set('region_name', 'CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


server_name pruthServer
tacc_compute_reservation_id 1ead9202-c138-45b9-b235-ce0bf59f9bcb
key_name pruth-jupyter
network_name pruthNet
image_name CC-CentOS7
flavor_name baremetal


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [19]:
chi.set('region_name', 'CHI@TACC')  

floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

Floating IP: 129.114.109.233


## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [17]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3291 , tacc_vlan  3504 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem


log4j:WARN No appenders could be found for logger (org.renci.ahab.libndl.LIBNDL).
log4j:WARN Please initialize the log4j system properly.


#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [20]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

uc_vlan  3291 , tacc_vlan  3504 , geni_pem  /home/pruth/work/geni-pruth.pem
Opening certificate /home/pruth/work/geni-pruth.pem and key /home/pruth/work/geni-pruth.pem
deleteSlice: uc-3291-tacc-3504

log4j:WARN No appenders could be found for logger (org.apache.commons.httpclient.HttpClient).
log4j:WARN Please initialize the log4j system properly.


### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [21]:
chi.set('region_name', 'CHI@TACC')

In [22]:
delete_server_by_name(server_name)

()

#### De-configure Network
TODO: break up into steps

In [27]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

{'network_id': '2d703759-ea1c-449c-ad88-38bb67c588be',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '81f39fde-8ad0-475e-a977-f0f2c3f2ec96',
 'subnet_ids': ['81f39fde-8ad0-475e-a977-f0f2c3f2ec96'],
 'port_id': 'c7b082dd-cca6-4b1c-8bbd-d142a31bd864',
 'id': '6337c0ed-60e0-4d66-b04f-ea9a137353cd'}

In [28]:
delete_router_by_name(router_name)

()

In [29]:
delete_subnet_by_name(subnet_name)

In [30]:
delete_network_by_name(network_name)

#### Release Lease

In [31]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id f6bdcb5a-f243-4506-895d-1a6ae5c7a42f


### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [32]:
chi.set('region_name', 'CHI@UC')

In [33]:
delete_server_by_name(server_name)

()

#### De-configure Network
TODO: break up into steps

In [35]:
detach_router_by_name(router_name=router_name, subnet_name=subnet_name)

{'network_id': 'e0fa08b8-ee90-4f5d-b4eb-11f60e46ca19',
 'tenant_id': 'd9faac3973a847f1b718fa765fe312e2',
 'subnet_id': '6563631a-90b5-499f-9ecf-7bc9e62aa2eb',
 'subnet_ids': ['6563631a-90b5-499f-9ecf-7bc9e62aa2eb'],
 'port_id': 'df921672-95d6-405e-b350-07593bd11c26',
 'id': '46bd0078-f99a-49d7-9996-f106a4c9c148'}

In [36]:
delete_router_by_name(router_name)

()

In [37]:
delete_subnet_by_name(subnet_name)

In [38]:
delete_network_by_name(network_name)

#### Release Lease

In [39]:
delete_lease_by_name(lease_name)

Deleted lease pruthLease with id 9f03cd52-2e42-45c2-b72a-d8605336cdef
